In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

# Load .env file
load_dotenv()

# Check key
if not os.getenv("GROQ_API_KEY"):
    raise ValueError("Missing GROQ_API_KEY in .env")

# Init Groq model
model = init_chat_model(
    "llama-3.3-70b-versatile",
    model_provider="groq",
    temperature=0.8
)

In [13]:
import json

sample_report = {
    "violations": [
        {"from_pin": "U1/Q", "to_pin": "U5/D", "slack": "-0.85 ns", "delay": "5.2 ns", "clock": "clk_main"},
        {"from_pin": "U2/Q", "to_pin": "U6/D", "slack": "-0.42 ns", "delay": "2.8 ns", "clock": "clk_aux"},
        {"from_pin": "U3/Q", "to_pin": "U7/D", "slack": "-1.15 ns", "delay": "6.0 ns", "clock": "clk_main"},
        {"from_pin": "U4/Q", "to_pin": "U8/D", "slack": "-0.25 ns", "delay": "3.0 ns", "clock": "clk_aux"},
        {"from_pin": "U9/Q", "to_pin": "U12/D", "slack": "-0.90 ns", "delay": "4.8 ns", "clock": "clk_main"}
    ]
}

with open("timing_features.json", "w") as f:
    json.dump(sample_report, f, indent=2)

print("Sample STA JSON report saved -> timing_features.json")


Sample STA JSON report saved -> timing_features.json


In [14]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import os

# 1. Load timing features
with open("timing_features.json", "r") as f:
    data = json.load(f)

violations = data.get("violations", [])

In [4]:
# 2. Define output schema
response_schemas = [
    ResponseSchema(name="root_cause", description="Root cause of violation"),
    ResponseSchema(name="suggestions", description="List of possible fixes")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt_timing = PromptTemplate.from_template(
    """
    ### TIMING VIOLATION ENTRY:
    Startpoint: {from_pin}
    Endpoint: {to_pin}
    Slack: {slack}
    Delay: {delay}
    Clock: {clock}

    ### INSTRUCTION:
    You are an expert VLSI STA assistant.
    Analyze the timing violation. Explain the root cause and suggest fixes (max 3).
    Return the output in JSON format with the following keys: `root_cause`, `suggestions`.
    Only return valid JSON.
    """
)

json_parser = JsonOutputParser()


In [9]:
with open("timing_features.json", "r") as f:
    data = json.load(f)

violations = data.get("violations", [])
results = []

for i, v in enumerate(violations, start=1):
    chain_timing = prompt_timing | model
    res = chain_timing.invoke({
        "from_pin": v["from_pin"],
        "to_pin": v["to_pin"],
        "slack": v["slack"],
        "delay": v["delay"],
        "clock": v["clock"]
    })
    
    parsed_res = json_parser.parse(res.content)
    results.append({"id": f"v{i}", **parsed_res})

# Save results
with open("llm_suggestions.json", "w") as f:
    json.dump({"violations": results}, f, indent=2)

print("Saved -> llm_suggestions.json")
print(json.dumps(results, indent=2))


Saved -> llm_suggestions.json
[
  {
    "id": "v1",
    "root_cause": "The timing violation is due to the excessive delay (5.2 ns) in the path from U1/Q to U5/D, which exceeds the available slack (-0.85 ns) for the clock 'clk_main', resulting in a negative slack that indicates a failure to meet the timing requirements.",
    "suggestions": [
      "Optimize the logic in the path from U1/Q to U5/D to reduce the delay, potentially by simplifying the combinational logic or pipelining the design.",
      "Increase the clock period of 'clk_main' to provide more time for the signal to propagate from U1/Q to U5/D, although this may impact the overall system performance.",
      "Insert a pipeline stage or a flip-flop in the path to break it into smaller segments, allowing the signal to propagate in multiple clock cycles and potentially mitigating the timing violation."
    ]
  }
]


In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.chat_models import ChatOpenAI
import json

# Few-shot examples
few_shot_examples = """
### EXAMPLES

Input:
Startpoint: U1/Q
Endpoint: U5/D
Slack: -0.85 ns
Delay: 5.2 ns
Clock: clk_main (period 5.0 ns)

Output:
{
  "root_cause": "Path delay exceeds clock period, causing setup violation.",
  "suggestions": [
    "Insert a pipeline register.",
    "Reduce combinational logic depth.",
    "Use faster cells on critical path."
  ]
}

Input:
Startpoint: U2/Q
Endpoint: U6/D
Slack: -0.4 ns
Delay: 2.8 ns
Clock: clk_aux (period 2.5 ns)

Output:
{
  "root_cause": "Path delay is slightly higher than the clock period, causing marginal setup violation.",
  "suggestions": [
    "Optimize logic in the critical path.",
    "Consider using faster gates.",
    "Check for unnecessary buffering."
  ]
}
"""


In [11]:
prompt_timing_with_example = PromptTemplate.from_template(
    f"""
### TIMING VIOLATION ENTRY:
Startpoint: {{from_pin}}
Endpoint: {{to_pin}}
Slack: {{slack}}
Delay: {{delay}}
Clock: {{clock}}

### INSTRUCTION:
You are an expert VLSI STA assistant.
Analyze the timing violation. Explain the root cause and suggest fixes (max 3).
Return the output in JSON format with the following keys: `root_cause`, `suggestions`.
Only return valid JSON.

{few_shot_examples}
"""
)

json_parser = JsonOutputParser()


In [12]:
with open("timing_features.json", "r") as f:
    data = json.load(f)

violations = data.get("violations", [])
results = []

for i, v in enumerate(violations, start=1):
    chain_timing = prompt_timing | model
    res = chain_timing.invoke({
        "from_pin": v["from_pin"],
        "to_pin": v["to_pin"],
        "slack": v["slack"],
        "delay": v["delay"],
        "clock": v["clock"]
    })
    
    parsed_res = json_parser.parse(res.content)
    results.append({"id": f"v{i}", **parsed_res})

# Save results
with open("llm_suggestions.json", "w") as f:
    json.dump({"violations": results}, f, indent=2)

print("Saved -> llm_suggestions.json")
print(json.dumps(results, indent=2))


Saved -> llm_suggestions.json
[
  {
    "id": "v1",
    "root_cause": "The timing violation is due to the delay of 5.2 ns between the startpoint U1/Q and the endpoint U5/D, which exceeds the available slack of -0.85 ns. This indicates that the signal is arriving too late, likely due to a combination of factors such as long combinational logic, high fanout, or insufficient clock buffering.",
    "suggestions": [
      "Optimize the combinational logic between U1/Q and U5/D to reduce the delay, potentially by breaking down complex logic into simpler stages or using faster logic cells.",
      "Insert a pipeline register between U1/Q and U5/D to break the long combinational path and reduce the delay, allowing the signal to propagate through the design more quickly.",
      "Review the clock tree synthesis and buffering to ensure that the clock signal is arriving at U1/Q and U5/D with sufficient timing margin, potentially by adding additional clock buffers or optimizing the clock tree topo

In [15]:
with open("timing_features.json", "r") as f:
    data = json.load(f)

violations = data.get("violations", [])
results = []

for i, v in enumerate(violations, start=1):
    chain_timing = prompt_timing | model
    res = chain_timing.invoke({
        "from_pin": v["from_pin"],
        "to_pin": v["to_pin"],
        "slack": v["slack"],
        "delay": v["delay"],
        "clock": v["clock"]
    })
    
    parsed_res = json_parser.parse(res.content)
    results.append({"id": f"v{i}", **parsed_res})

# Save results
with open("llm_suggestions.json", "w") as f:
    json.dump({"violations": results}, f, indent=2)

print("Saved -> llm_suggestions.json")
print(json.dumps(results, indent=2))


Saved -> llm_suggestions.json
[
  {
    "id": "v1",
    "root_cause": "The timing violation is caused by the delay on the path from U1/Q to U5/D exceeding the available clock period, resulting in a negative slack of -0.85 ns. This indicates that the signal is arriving at the endpoint U5/D 0.85 ns after the expected clock edge, likely due to a combination of cell delays, wire delays, and clock skew.",
    "suggestions": [
      "Optimize the logic between U1/Q and U5/D to reduce the cell delays, potentially by simplifying the combinational logic or using faster cell variants.",
      "Investigate the wire delays and consider using a shorter route or adding buffer cells to reduce the wire length and capacitance.",
      "Review the clock tree synthesis and consider rebalancing the clock tree to reduce clock skew and ensure that the clock signal arrives at U5/D earlier, increasing the available setup time."
    ]
  },
  {
    "id": "v2",
    "root_cause": "The timing violation is caused b